In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
indexing_dataset_search = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
indexing_dataset_search.indices.close(index='dataset_search')
indexing_dataset_search.indices.put_settings(index='dataset_search', body={"index": {"similarity": {"default": {"type": "classic"}}}})
indexing_dataset_search.indices.open(index='dataset_search')

In [ ]:
def elasticsearch_search(query):
    
    result= indexing_dataset_search.search(
        index="dataset_search", 
        body = {
        "_source": ["dataset_id","dataset_title"],
        "from" : 0,
        "size" : 100,
        "query": {
            "multi_match":{
              "type": "most_fields",
              "query":    query, 
              "fields": ["dataset_title", "dataset_description"] 
            }
        }
    })
    
    return result

In [ ]:
def return_data_index(query):
    
    datasets_index = []

    result_index = elasticsearch_search(query)
        
    for hit in result_index['hits']['hits']:
    
        dataset_id = hit['_source']['dataset_id']
        
        dataset_title = hit['_source']['dataset_title']
    
        datasets_index.append(dataset_id)
    
    return datasets_index

In [ ]:
relevance_judgments = pd.read_csv('../../data/data_search_2_e_train_qrels.txt', delimiter=" ")
relevance_judgments = pd.DataFrame(relevance_judgments.values, columns = ["query_id", "dataset_id", "relevance"])

In [ ]:
def get_relevance(query_id, dataset_id):
    
    relevance_by_query = relevance_judgments.loc[relevance_judgments['query_id'] == query_id]
    
    relevance_by_dataset = relevance_by_query.loc[relevance_by_query['dataset_id'] == dataset_id]
    
    if relevance_by_dataset.empty:
    
        return 0
    
    else:
    
        relevance_jugdment = relevance_by_dataset['relevance'].values[0].replace("L","")
        
        return int(relevance_jugdment)

In [ ]:
train_queries = pd.read_csv('../../data/train_queries.csv')

In [ ]:
train_queries.sample(5)

In [ ]:
result_recall = []

for i,row in tqdm(train_queries.iterrows()):
    
    query_id = row['query_id']
    query_description = row['query_description']
    
    result_index = return_data_index(query_description)
    
    #getting the relevance score for each elasticsearch result
    predict_relevance = []
    
    for dataset_id in result_index:
        
        value_relevance = get_relevance(query_id,dataset_id)
    
        predict_relevance.append(value_relevance)
    
    ##getting the relevant datasets for the query
    relevance_by_query = relevance_judgments.loc[relevance_judgments['query_id'] == query_id]
    relevant_datasets = relevance_by_query[(relevance_by_query['relevance'] == 'L2') | (relevance_by_query['relevance'] == 'L1')]
    
    #computing the recall
    relevant_predict = np.count_nonzero(predict_relevance)
    
    correct_relevant = relevant_datasets["query_id"].count()
    
    #print("predict = "+str(relevant_predict)+" relevant ="+str(correct_relevant))
    
    if (correct_relevant != 0):
        
        recall = relevant_predict/correct_relevant
    
    else:
        
        recall = 0
        
    result_recall.append(recall)

In [ ]:
np.mean(result_recall)